# Macro Impact Tracker – Market Price Reactions

This notebook measures how financial markets react in the minutes and hours
following macroeconomic releases in India and the United States.

We align macro event timestamps with intraday market prices
to compute post-event returns.


In [ ]:
import pandas as pd
import yfinance as yf
from datetime import timedelta


In [ ]:
macro_events = pd.read_csv("../data/macro_events_all.csv")
macro_events["datetime"] = pd.to_datetime(macro_events["datetime"])

macro_events.head()


In [ ]:
ASSETS = {
    "US": {
        "SPY": "SPY",
        "VIX": "^VIX",
        "US10Y": "^TNX"
    },
    "INDIA": {
        "NIFTY50": "^NSEI",
        "USDINR": "USDINR=X",
        "INDIAVIX": "^INDIAVIX"
    }
}


In [ ]:
def fetch_intraday_prices(ticker, start, end, interval="5m"):
    data = yf.download(
        ticker,
        start=start,
        end=end,
        interval=interval,
        progress=False
    )
    
    if data.empty:
        return None
    
    data = data.reset_index()
    data.rename(columns={"Datetime": "datetime"}, inplace=True)
    return data[["datetime", "Close"]]


In [ ]:
# Define cutoff for intraday availability (last 45 days)
INTRADAY_LOOKBACK_DAYS = 45

latest_date = macro_events["datetime"].max()
intraday_cutoff = latest_date - pd.Timedelta(days=INTRADAY_LOOKBACK_DAYS)

recent_events = macro_events[macro_events["datetime"] >= intraday_cutoff]
historical_events = macro_events[macro_events["datetime"] < intraday_cutoff]

len(recent_events), len(historical_events)




In [ ]:
def fetch_daily_prices(ticker, start, end):
    data = yf.download(
        ticker,
        start=start,
        end=end,
        interval="1d",
        progress=False
    )

    if data.empty:
        return None

    data = data.reset_index()
    data.rename(columns={"Date": "date"}, inplace=True)
    return data[["date", "Close"]]


In [ ]:
def fetch_intraday_prices(ticker, start, end):
    data = yf.download(
        ticker,
        start=start,
        end=end,
        interval="5m",
        progress=False
    )

    if data.empty:
        return None

    data = data.reset_index()
    data.rename(columns={"Datetime": "datetime"}, inplace=True)
    return data[["datetime", "Close"]]


In [ ]:
test_daily = fetch_daily_prices(
    ticker="^NSEI",
    start="2022-01-01",
    end="2022-12-31"
)

test_daily.head(100)


In [ ]:
results = []

for _, event in historical_events.iterrows():
    country = event["country"]
    event_time = event["datetime"]
    event_date = event_time.date()

    # Choose asset
    ticker = "^NSEI" if country == "INDIA" else "SPY"

    prices = fetch_daily_prices(
        ticker=ticker,
        start=event_date - timedelta(days=5),
        end=event_date + timedelta(days=5)
    )

    if prices is None:
        continue

    prices["date"] = pd.to_datetime(prices["date"]).dt.date

    event_day = prices[prices["date"] == event_date]
    prev_day = prices[prices["date"] < event_date].tail(1)
    next_day = prices[prices["date"] > event_date].head(1)

    if event_day.empty or prev_day.empty or next_day.empty:
        continue

    event_close = float(event_day["Close"].iloc[0])
    prev_close = float(prev_day["Close"].iloc[0])
    next_close = float(next_day["Close"].iloc[0])

    results.append({
        "country": country,
        "event": event["event"],
        "event_datetime": event_time,
        "asset": ticker,
        "event_day_return_pct": (event_close / prev_close - 1) * 100,
        "next_day_return_pct": (next_close / event_close - 1) * 100
    })

daily_reactions = pd.DataFrame(results)
daily_reactions.head()


In [ ]:
from datetime import datetime, timedelta

# -------------------------------------------------------
# Intraday Macro Impact Analysis (Production-safe)
# -------------------------------------------------------

# Work on a clean copy to avoid SettingWithCopyWarning
recent_events_clean = recent_events.copy()
recent_events_clean["datetime"] = pd.to_datetime(
    recent_events_clean["datetime"]
).dt.tz_localize(None)

today = pd.Timestamp.utcnow().tz_localize(None)
MAX_INTRADAY_LOOKBACK = 60  # Yahoo Finance limit (days)

intraday_results = []

for _, event in recent_events_clean.iterrows():
    event_time = event["datetime"]

    # Skip events outside Yahoo intraday availability
    if (today - event_time).days > MAX_INTRADAY_LOOKBACK:
        continue

    country = event["country"]
    ticker = "^NSEI" if country == "INDIA" else "SPY"

    start = event_time - timedelta(hours=1)
    end = event_time + timedelta(hours=2)

    prices = yf.download(
        ticker,
        start=start,
        end=end,
        interval="5m",
        progress=False
    )

    if prices.empty:
        continue

    prices = prices.reset_index()
    prices.rename(columns={"Datetime": "datetime"}, inplace=True)
    prices["datetime"] = pd.to_datetime(prices["datetime"]).dt.tz_localize(None)

    before_event = prices[prices["datetime"] <= event_time]
    after_event = prices[prices["datetime"] > event_time]

    if before_event.empty or after_event.empty:
        continue

    base_price = float(before_event["Close"].iloc[-1])

    def compute_return(minutes):
        target_time = event_time + timedelta(minutes=minutes)
        window = after_event[after_event["datetime"] <= target_time]
        if window.empty:
            return None
        return (float(window["Close"].iloc[-1]) / base_price - 1) * 100

    intraday_results.append({
        "country": country,
        "event": event["event"],
        "event_datetime": event_time,
        "asset": ticker,
        "return_5m_pct": compute_return(5),
        "return_30m_pct": compute_return(30),
        "return_1h_pct": compute_return(60)
    })

intraday_reactions = pd.DataFrame(intraday_results)
intraday_reactions


In [ ]:
intraday_reactions.shape


In [36]:
# Save outputs for dashboard
daily_reactions.to_csv("../data/daily_reactions.csv", index=False)
intraday_reactions.to_csv("../data/intraday_reactions.csv", index=False)


In [37]:
!ls ../data


daily_reactions.csv  india_reporate.csv      macro_events_all.csv
india_cpi.csv	     intraday_reactions.csv
